In [12]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import airy, airye, spherical_jn as sph_jn, spherical_yn as sph_yn
import os

dirs = [n for n in os.listdir() if (os.path.isdir(n) and (n[-2:]=='_s'))]
for dir in dirs:
    r, hls, ddrls = load_matrices(dir)
    print(dir)
    emin = hls[-1,0,0]
    n = len(hls[0])
    shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
    vd = (hls-shift)/4.637

    hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
    emax = np.abs(np.array(hfunc(r[0])[-1,-1]))

    w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
    wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])

    es = np.array([1])
    for i in range(n):
        emax = np.abs(np.array(hfunc(r[0])[i,i]))
        emin = np.abs(np.array(hfunc(r[-1])[i,i]))
        es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
    es = np.unique(np.round(np.sort(es)))
    sigmas = np.zeros((len(es), n, n))
    ecm = es*4.637+np.real(hls[-1,0,0])
    for ei in range(len(es)):
        e = es[ei]
        lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
        ks = np.diag(np.sqrt(e*np.eye(n)-hfunc(r[-1])))
        yf = np.zeros((lmax, n, n), dtype=complex)
        nt = np.sum(np.diag(vd[-1]) < e)
        for l in range(lmax):
            wtest = w(r[:-1]+np.diff(r)/2, l, e)
            wptest = wp(r[:-1]+np.diff(r)/2, l)
            k2 = np.zeros((len(wtest), n))
            t = np.zeros((len(wtest), n, n), dtype=complex)
            wpt = np.zeros((len(wtest), n, n), dtype=complex)
            for i in range(len(wtest)):
                k2[i], t[i] = np.linalg.eigh(wtest[i])
                wpt[i] = np.matrix(t[i]).H @ (np.matrix(wptest[i]) @ np.matrix(t[i]))
            p = np.array([np.matrix(t[i]).H @ np.matrix(t[i-1]) if i > 0 else np.matrix(t[0]) for i in range(len(wtest))])
            c1 = np.zeros((len(wtest), n, n), dtype=complex)
            c2 = np.zeros((len(wtest), n, n), dtype=complex)
            c3 = np.zeros((len(wtest), n, n), dtype=complex)
            c4 = np.zeros((len(wtest), n, n), dtype=complex)
            for i in range(len(wtest)):
                phase = 2*np.pi/3
                alpha = np.array([-(wpt[i,j,j])**(1./3.) if np.isreal(-(wpt[i,j,j])**(1./3.)) else -(wpt[i,j,j])**(1./3.)*np.exp(1j*phase) for j in range(n)])
                beta = k2[i]/np.diag(wpt[i])
                wr = alpha/np.pi
                rhob = np.diff(r)[i]/2
                x = np.array([[alpha[j]*(-rhob+beta[j]), alpha[j]*(rhob+beta[j])] for j in range(n)])
                for j in range(n):
                    if np.any(np.real(x[j]) > 100):
                        dksi = np.diff((2./3)*(x[j]**(3./2)))[0]
                        aie, aiep, bie, biep = airye(x[j])
                        c1[i,j,j] = np.pi*(np.exp(-dksi)*aie[1]*biep[0]-np.exp(dksi)*bie[1]*aiep[0]) #/wr[j]
                        c2[i,j,j] = np.pi*(np.exp(dksi)*bie[1]*aie[0]-np.exp(-dksi)*aie[1]*bie[0])/alpha[j]
                        c3[i,j,j] = np.pi*alpha[j]*(np.exp(-dksi)*aiep[1]*biep[0]-np.exp(dksi)*biep[1]*aiep[0]) #/wr[j]
                        c4[i,j,j] = np.pi*(np.exp(dksi)*biep[1]*aie[0]-np.exp(-dksi)*aiep[1]*bie[0]) #/wr[j]
                    else:
                        ai, aip, bi, bip = airy(x[j])
                        c1[i,j,j] = np.pi*(ai[1]*bip[0]-bi[1]*aip[0]) #/wr[j]
                        c2[i,j,j] = np.pi*(bi[1]*ai[0]-ai[1]*bi[0])/alpha[j]
                        c3[i,j,j] = np.pi*alpha[j]*(aip[1]*bip[0]-bip[1]*aip[0]) #/wr[j]
                        c4[i,j,j] = np.pi*(bip[1]*ai[0]-aip[1]*bi[0]) #/wr[j]
            y = np.zeros((len(r), n, n), dtype=complex)
            y[0] = 1e308*np.eye(n, dtype=complex)
            for i in range(len(wtest)):
                try:
                    prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
                except np.linalg.LinAlgError:
                    print(np.matrix(c1[i] + c2[i] @ y[i]))
                # print(prey)
                y[i+1] = np.matrix(p[i]) @ (prey @ np.matrix(p[i]).H)
                # print(y[i+1])
            yf[l] = np.matrix(t[-1]).H @ (y[-1] @ np.matrix(t[-1]))
        jmat = np.array([np.diag([sph_jn(int(l), k*r[-1]) for k in ks]) for l in range(lmax)])
        nmat = np.array([np.diag([sph_yn(int(l), k*r[-1]) for k in ks]) for l in range(lmax)])
        jpmat = np.array([np.diag([k*sph_jn(int(l), k*r[-1], derivative=True) for k in ks]) for l in range(lmax)])
        npmat = np.array([np.diag([k*sph_yn(int(l), k*r[-1], derivative=True) for k in ks]) for l in range(lmax)])
        rmat = np.array([-np.matrix(npmat[l] - yf[l] @ nmat[l]).I @ np.matrix(jpmat[l] - yf[l] @ jmat[l]) for l in range(lmax)])
        smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
        sigmas[ei,:nt,:nt] = np.pi * 1e-16 * np.sum(np.array([(2*l+1)*(np.abs(smat[l]-np.eye(nt))**2) for l in range(lmax)]),axis=0) @ np.matrix(np.abs(np.diag(ks[:nt]))**2).I
        print(ecm[ei])
        print(sigmas[ei])
    headerfinal = 'E, cm-1\t'
    for i in range(n):
        for j in range(n):
            headerfinal += f'Sigma_{i}{j}, cm2\t'
    tofile = np.zeros((len(es), n*n+1))
    tofile[:,0] = ecm
    for i in range(n):
        for j in range(n):
            tofile[:,i*n+j+1] = sigmas[:,i,j]

    np.savetxt(f'{dir}/sigmas_total_airy.txt', tofile, fmt='%.6e', delimiter='\t', header=headerfinal, comments='')
    print(f'Directory {dir} is done!')

H_1_s
93087.297
[[3.18660301e-14 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
93091.93400000001
[[3.45700137e-14 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
93096.571
[[3.46468285e-14 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
93101.208
[[3.21363592e-14 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
93105.845
[[3.37468777e-14 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
93110.482
[[3.1899215e-14 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00]]
93115.119
[[3.16508143e-14 0.00000000e+00 0.00000000e+00]
 [0.000

/tmp/ipykernel_53203/161557389.py:75: RuntimeWarning: overflow encountered in matmul
  prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
/tmp/ipykernel_53203/161557389.py:75: RuntimeWarning: invalid value encountered in matmul
  prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)


96212.63500000001
[[2.45022273e-14 2.27675466e-16 1.91272555e-20]
 [1.53940194e-16 2.63946350e-14 2.18107937e-20]
 [2.19068617e-21 3.69456390e-21 2.81003970e-14]]
96217.272
[[2.44837796e-14 2.29093142e-16 1.91546794e-20]
 [1.54997806e-16 2.63769714e-14 2.20115876e-20]
 [2.20653365e-21 3.74777553e-21 2.80969673e-14]]
96282.19
[[2.42156021e-14 2.49006435e-16 1.91834768e-20]
 [1.69948747e-16 2.61278889e-14 2.29464447e-20]
 [2.38783514e-21 4.18489947e-21 2.80460502e-14]]
96291.464
[[2.41767687e-14 2.51842029e-16 1.91705867e-20]
 [1.72093215e-16 2.60932379e-14 2.27944819e-20]
 [2.41160008e-21 4.19627627e-21 2.80353323e-14]]
96356.382
[[2.39163943e-14 2.71611458e-16 1.92918628e-20]
 [1.87149321e-16 2.58315784e-14 2.06437457e-20]
 [2.60513653e-21 4.04580229e-21 2.79917988e-14]]
96365.656
[[2.38778011e-14 2.74435916e-16 1.93681370e-20]
 [1.89314221e-16 2.57942719e-14 2.03199011e-20]
 [2.64093262e-21 4.01650924e-21 2.79880100e-14]]
96435.21100000001
[[2.35957588e-14 2.95402076e-16 2.03440173e-2

/tmp/ipykernel_53203/161557389.py:79: RuntimeWarning: invalid value encountered in matmul
  y[i+1] = np.matrix(p[i]) @ (prey @ np.matrix(p[i]).H)


96285.054
[[2.47387909e-14]]
96359.246
[[2.44532563e-14]]
96433.438
[[2.427831e-14]]
96512.26699999999
[[2.39840384e-14]]
96595.733
[[2.36751463e-14]]
96674.562
[[2.33886455e-14]]
96758.028
[[2.30853038e-14]]
96846.131
[[2.27669803e-14]]
96934.234
[[2.24550113e-14]]
97022.337
[[2.22591367e-14]]
97115.077
[[2.19404914e-14]]
97207.817
[[2.16260151e-14]]
97305.194
[[2.13027142e-14]]
97402.571
[[2.09852516e-14]]
97504.585
[[2.06606086e-14]]
97606.599
[[2.04559432e-14]]
97713.25
[[2.01286039e-14]]
97819.901
[[1.98100177e-14]]
97931.189
[[1.94831076e-14]]
98047.114
[[1.91534324e-14]]
98158.402
[[1.89543946e-14]]
98278.964
[[1.86280721e-14]]
98399.526
[[1.83084503e-14]]
98524.725
[[1.7985742e-14]]
98649.924
[[1.77833629e-14]]
98779.76
[[1.74645569e-14]]
98914.233
[[1.71471496e-14]]
99048.706
[[1.6836064e-14]]
99187.816
[[1.66332261e-14]]
99331.563
[[1.6325606e-14]]
99475.31
[[1.60232737e-14]]
Directory H_2_s is done!
H_0_minus_s
93089.497
[[3.11244387e-14 0.00000000e+00]
 [0.00000000e+00 0.00

In [14]:
dirp = [n for n in os.listdir() if (os.path.isdir(n) and (n[-2:]=='_p'))]
dirp

['H_0_minus_p', 'H_3_p', 'H_0_plus_p', 'H_1_p', 'H_2_p']